In [1]:
%%time
import asyncio
from requests_html import AsyncHTMLSession
asession = AsyncHTMLSession()
import json
import pprint
import pandas as pd
import numpy as np
import re
import os

CPU times: total: 1.61 s
Wall time: 1.97 s


In [2]:
# Fetching data through API request about main page where all category codes are kept
r = await asession.get('https://www.zara.com/in/en/categories?ajax=true')
r.status_code

200

In [3]:
json_string = r.text
data = json.loads(json_string)
# List to collect all the category codes
my_list = []

In [4]:
#Parsing category codes and their all member names from json response of main page 
# creating urls for each page of the site

def invoke(id,name,sec_name,name2, name1):
    if sec_name == 'BEAUTY':
        url = f"https://www.zara.com/in/en/{name1}-l{name2}.html?v1={id}&regionGroupId=69&ajax=true"
        my_list.append([id,name,sec_name, "Cosmetics", "Accessories",url])
    else:
        url = f"https://www.zara.com/in/en/category/{id}/products?regionGroupId=69&ajax=true"
        my_list.append([id,name,sec_name,name2, name1, url])

def fetch(j):
    id =j.get('id')
    name = j.get('name')
    sec_name = j.get("sectionName")
    subcat = j.get("subcategories", [])
    seo_id = j.get("seo", {}).get("seoCategoryId")
    keyword = j.get("seo", {}).get("keyword")
    if sec_name == 'BEAUTY':
        print(keyword)
        print(seo_id)
        return id,name,sec_name,seo_id,keyword            
    else:
        return id,name,sec_name,subcat


cat = data['categories']    
for i in cat:
    sub_1 = i.get('subcategories', [])
    for j in sub_1:
        name1 = 0
        name2 = 0
        drift = j.get("sectionName")
        if(drift == 'BEAUTY'):
            id,name,sec_name,seo_id,keyword = fetch(j)
            if (keyword != None) and (seo_id != None):
                invoke(id,name,sec_name,seo_id,keyword)
            else:
                break
        else:
            id,name,sec_name,sub_2 = fetch(j)
            invoke(id,name,sec_name,name2, name1)
            name1 = name   
            if sub_2 != []:
                for l in sub_2:
                    id,name,sec_name,sub_3 = fetch(l)
                    invoke(id,name,sec_name,name2, name1)
                    name2 = name
                    if sub_3 != []:                                    
                        for m in sub_3:
                            id,name,sec_name,sub_4 = fetch(m)
                            invoke(id,name,sec_name, name2, name1)      

woman-beauty-makeup
4414
woman-beauty-perfume-best
4882
woman-beauty-lips
4238
woman-beauty-face
4240
woman-beauty-nails
4241
woman-beauty-tools
4242
woman-beauty-tutorials
6464
woman-beauty-shade-finder
5079
None
None


In [5]:
%%time
print(len(my_list))
df = pd.DataFrame(my_list, columns=['category_id', 'Category', 'Section', 'Sub_branch', 'Main_branch', 'URL'])


1365
CPU times: total: 0 ns
Wall time: 13 ms


In [6]:
df.head(30)

,category_id,Category,Section,Sub_branch,Main_branch,URL
0,2290613,NEW,WOMAN,0,0,https://www.zara.com/in/en/category/2290613/pr...
1,2329646,__,WOMAN,0,0,https://www.zara.com/in/en/category/2329646/pr...
2,1899786,DIVIDER_MENU_WOMAN1,WOMAN,0,0,https://www.zara.com/in/en/category/1899786/pr...
3,2290795,DRESSES | JUMPSUITS,WOMAN,0,0,https://www.zara.com/in/en/category/2290795/pr...
4,2290847,View all,WOMAN,0,DRESSES | JUMPSUITS,https://www.zara.com/in/en/category/2290847/pr...
5,2290837,Midi | Maxi,WOMAN,View all,DRESSES | JUMPSUITS,https://www.zara.com/in/en/category/2290837/pr...
6,2290794,Short,WOMAN,Midi | Maxi,DRESSES | JUMPSUITS,https://www.zara.com/in/en/category/2290794/pr...
7,2290830,GOING OUT,WOMAN,Short,DRESSES | JUMPSUITS,https://www.zara.com/in/en/category/2290830/pr...
8,2290827,Jumpsuits,WOMAN,GOING OUT,DRESSES | JUMPSUITS,https://www.zara.com/in/en/category/2290827/pr...
9,2290820,Long Sleeve,WOMAN,Jumpsuits,DRESSES | JUMPSUITS,https://www.zara.com/in/en/category/2290820/pr...


In [7]:
# Create a copy of rows with None values before filtering
rows_with_none = df[df['Section'].isnull()].copy()

# Filter rows in 'column2' where None values are removed
df_cleaned = df.dropna(subset=['Section']).reset_index(drop=True)


In [8]:
def replace_zero(row):
    regex = r'view all|viewall'
    # Using re.search() to find if the pattern matches the 'Category' column value
    if re.search(regex, row['Category'], re.IGNORECASE):
        if row['Sub_branch'] == 0:
            return row['Main_branch']
        else:
            return row['Sub_branch']
    else:
        return row['Category']  # Correcting the column name spelling here

# Apply the function to replace values in 'Category' column based on regex pattern
df_cleaned['Category'] = df_cleaned.apply(replace_zero, axis=1)

In [9]:
df_cleaned.head(30)

,category_id,Category,Section,Sub_branch,Main_branch,URL
0,2290613,NEW,WOMAN,0,0,https://www.zara.com/in/en/category/2290613/pr...
1,2329646,__,WOMAN,0,0,https://www.zara.com/in/en/category/2329646/pr...
2,1899786,DIVIDER_MENU_WOMAN1,WOMAN,0,0,https://www.zara.com/in/en/category/1899786/pr...
3,2290795,DRESSES | JUMPSUITS,WOMAN,0,0,https://www.zara.com/in/en/category/2290795/pr...
4,2290847,DRESSES | JUMPSUITS,WOMAN,0,DRESSES | JUMPSUITS,https://www.zara.com/in/en/category/2290847/pr...
5,2290837,Midi | Maxi,WOMAN,View all,DRESSES | JUMPSUITS,https://www.zara.com/in/en/category/2290837/pr...
6,2290794,Short,WOMAN,Midi | Maxi,DRESSES | JUMPSUITS,https://www.zara.com/in/en/category/2290794/pr...
7,2290830,GOING OUT,WOMAN,Short,DRESSES | JUMPSUITS,https://www.zara.com/in/en/category/2290830/pr...
8,2290827,Jumpsuits,WOMAN,GOING OUT,DRESSES | JUMPSUITS,https://www.zara.com/in/en/category/2290827/pr...
9,2290820,Long Sleeve,WOMAN,Jumpsuits,DRESSES | JUMPSUITS,https://www.zara.com/in/en/category/2290820/pr...


In [10]:
# Function to replace zero with another column's value
def replace_zero(row):
    if row['Main_branch'] == 0:
        return row['Category']
    else:
        return row['Main_branch']

# Apply the function to replace zero values in 'column_name'
df_cleaned['Main_branch'] = df_cleaned.apply(replace_zero, axis=1)

In [11]:
f = df_cleaned[df_cleaned['Section'] == 'KID']
f.head(30)

,category_id,Category,Section,Sub_branch,Main_branch,URL
434,2118261,PARTY,KID,0,PARTY,https://www.zara.com/in/en/category/2118261/pr...
435,2118325,GIFT GUIDE,KID,0,GIFT GUIDE,https://www.zara.com/in/en/category/2118325/pr...
436,2118326,Editorial,KID,0,GIFT GUIDE,https://www.zara.com/in/en/category/2118326/pr...
437,2118327,Collection,KID,Editorial,GIFT GUIDE,https://www.zara.com/in/en/category/2118327/pr...
438,2347139,Collection,KID,Collection,GIFT GUIDE,https://www.zara.com/in/en/category/2347139/pr...
439,2345654,6-14 YEARS | GIRL,KID,Collection,GIFT GUIDE,https://www.zara.com/in/en/category/2345654/pr...
440,2345655,1-6 YEARS | GIRL,KID,Collection,GIFT GUIDE,https://www.zara.com/in/en/category/2345655/pr...
441,2345656,6-14 YEARS | BOY,KID,Collection,GIFT GUIDE,https://www.zara.com/in/en/category/2345656/pr...
442,2345657,1-6 YEARS | BOY,KID,Collection,GIFT GUIDE,https://www.zara.com/in/en/category/2345657/pr...
443,2345658,0 - 18 MONTHS | BABY,KID,Collection,GIFT GUIDE,https://www.zara.com/in/en/category/2345658/pr...


In [12]:
df_cleaned.to_csv('Zara_main_urls.csv', index=False)

In [13]:
# Fetching responses from all urls made using category codes and store them in files

product_list = []
links = []
urls = df_cleaned['URL']
id = df_cleaned['category_id']

for j in range(1350):     
    val = urls[j]
    c = id[j]
    print(val,c)
    # Collecting all responses of the urls
    r = await asession.get(val)
    num = r.status_code
    if(num == 200):                    
        try:
            r.raise_for_status()  # Raise an error for bad HTTP responses
            json_string = r.text
            data = json.loads(json_string)
            links.append(c) 
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON: {e}")
        except Exception as e:
            print(f"HTTP request error: {e}")
        product_list.append(data)            

https://www.zara.com/in/en/category/2290613/products?regionGroupId=69&ajax=true 2290613
https://www.zara.com/in/en/category/2329646/products?regionGroupId=69&ajax=true 2329646
https://www.zara.com/in/en/category/1899786/products?regionGroupId=69&ajax=true 1899786
https://www.zara.com/in/en/category/2290795/products?regionGroupId=69&ajax=true 2290795
https://www.zara.com/in/en/category/2290847/products?regionGroupId=69&ajax=true 2290847
https://www.zara.com/in/en/category/2290837/products?regionGroupId=69&ajax=true 2290837
https://www.zara.com/in/en/category/2290794/products?regionGroupId=69&ajax=true 2290794
https://www.zara.com/in/en/category/2290830/products?regionGroupId=69&ajax=true 2290830
https://www.zara.com/in/en/category/2290827/products?regionGroupId=69&ajax=true 2290827
https://www.zara.com/in/en/category/2290820/products?regionGroupId=69&ajax=true 2290820
https://www.zara.com/in/en/category/2290866/products?regionGroupId=69&ajax=true 2290866
https://www.zara.com/in/en/categ

KeyError: 1320

In [14]:
# Approximately 1350 responses
output_directory = "C:\Json files of zara 1300+responses"

os.makedirs(output_directory, exist_ok=True)

# Save each JSON object in a separate file
for i, json_data in enumerate(product_list):
    file_path = os.path.join(output_directory, f'zarajson_{i}_{links[i]}.json')
    with open(file_path, 'w') as json_file:
        json.dump(json_data, json_file, indent=2)

print(f"All {len(product_list)} JSON files have been saved to {output_directory}")

All 1318 JSON files have been saved to C:\Json files of zara 1300+responses


In [15]:
%%time

# Parsing each json response one by one to extract all products from it and saving all of it
print(len(links))

def call(i):
    file_path = f"C:\Json files of zara 1300+responses\zarajson_{i}_{links[i]}.json"
    
    # Read the JSON file
    with open(file_path, 'r') as json_file:
        response = json.load(json_file)    
    # Process the JSON data as needed
    print(f"Processing {file_path}:")
    return response

# A generator parsing each response and then storing them into individual files
def extract_info_recursive_generator(data):
    if isinstance(data, list):
        for item in data:
            yield from extract_info_recursive_generator(item)
    elif isinstance(data, dict):
        xmedia = data.get("commercialComponents", [])
        for entry in xmedia:
            yield {
                "name": entry.get("name"),
                "price": entry.get("price"),
                "sectionName": entry.get("sectionName"),
                "productId": entry.get("detail", {}).get("colors", [])[0].get("productId"),
                "color": entry.get("detail", {}).get("colors", [])[0].get("name"),
                "keyword" : entry.get("seo", {}).get("keyword"),
            }

        for value in data.values():
            yield from extract_info_recursive_generator(value)

for i in range(1400):
    data = call(i)
    extracted_info_generator = extract_info_recursive_generator(data)
    extracted_info_list = list(extracted_info_generator)

    output_directory = "C:\\2nd Zara Product outputs"
    os.makedirs(output_directory, exist_ok=True)

    # Save the extracted information to a separate JSON file for each input file
    output_file_path = os.path.join(output_directory, f"parsed_output_{i}.json")
    with open(output_file_path, 'w') as output_file:
        json.dump(extracted_info_list, output_file)


1318
Processing C:\Json files of zara 1300+responses\zarajson_0_2290613.json:
Processing C:\Json files of zara 1300+responses\zarajson_1_2329646.json:
Processing C:\Json files of zara 1300+responses\zarajson_2_1899786.json:
Processing C:\Json files of zara 1300+responses\zarajson_3_2290795.json:
Processing C:\Json files of zara 1300+responses\zarajson_4_2290847.json:
Processing C:\Json files of zara 1300+responses\zarajson_5_2290837.json:
Processing C:\Json files of zara 1300+responses\zarajson_6_2290794.json:
Processing C:\Json files of zara 1300+responses\zarajson_7_2290830.json:
Processing C:\Json files of zara 1300+responses\zarajson_8_2290827.json:
Processing C:\Json files of zara 1300+responses\zarajson_9_2290820.json:
Processing C:\Json files of zara 1300+responses\zarajson_10_2290866.json:
Processing C:\Json files of zara 1300+responses\zarajson_11_2290803.json:
Processing C:\Json files of zara 1300+responses\zarajson_12_2290793.json:
Processing C:\Json files of zara 1300+respo

IndexError: list index out of range

In [16]:
# Processing every output file and combining all of it into one single large dataframe
result_df = pd.DataFrame()
for i in range(1400):        
    # Specify the directory where the JSON files are saved
    file_path = f"C:\\2nd Zara Product outputs\\parsed_output_{i}.json"
    # Read the JSON file
    with open(file_path, 'r') as json_file:
        response = json.load(json_file)    
    # Process the JSON data as needed
    print(f"Processing {file_path}:")
    df = pd.DataFrame(response)
    df['category_id'] = links[i]
    result_df = pd.concat([result_df, df], ignore_index=True, sort=False)

Processing C:\2nd Zara Product outputs\parsed_output_0.json:
Processing C:\2nd Zara Product outputs\parsed_output_1.json:
Processing C:\2nd Zara Product outputs\parsed_output_2.json:
Processing C:\2nd Zara Product outputs\parsed_output_3.json:
Processing C:\2nd Zara Product outputs\parsed_output_4.json:
Processing C:\2nd Zara Product outputs\parsed_output_5.json:
Processing C:\2nd Zara Product outputs\parsed_output_6.json:
Processing C:\2nd Zara Product outputs\parsed_output_7.json:
Processing C:\2nd Zara Product outputs\parsed_output_8.json:
Processing C:\2nd Zara Product outputs\parsed_output_9.json:
Processing C:\2nd Zara Product outputs\parsed_output_10.json:
Processing C:\2nd Zara Product outputs\parsed_output_11.json:
Processing C:\2nd Zara Product outputs\parsed_output_12.json:
Processing C:\2nd Zara Product outputs\parsed_output_13.json:
Processing C:\2nd Zara Product outputs\parsed_output_14.json:
Processing C:\2nd Zara Product outputs\parsed_output_15.json:
Processing C:\2nd 

C:\Users\madan\AppData\Local\Temp\ipykernel_772\2859615535.py:13: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result_df = pd.concat([result_df, df], ignore_index=True, sort=False)


Processing C:\2nd Zara Product outputs\parsed_output_386.json:
Processing C:\2nd Zara Product outputs\parsed_output_387.json:
Processing C:\2nd Zara Product outputs\parsed_output_388.json:
Processing C:\2nd Zara Product outputs\parsed_output_389.json:
Processing C:\2nd Zara Product outputs\parsed_output_390.json:
Processing C:\2nd Zara Product outputs\parsed_output_391.json:
Processing C:\2nd Zara Product outputs\parsed_output_392.json:
Processing C:\2nd Zara Product outputs\parsed_output_393.json:
Processing C:\2nd Zara Product outputs\parsed_output_394.json:
Processing C:\2nd Zara Product outputs\parsed_output_395.json:
Processing C:\2nd Zara Product outputs\parsed_output_396.json:
Processing C:\2nd Zara Product outputs\parsed_output_397.json:
Processing C:\2nd Zara Product outputs\parsed_output_398.json:
Processing C:\2nd Zara Product outputs\parsed_output_399.json:
Processing C:\2nd Zara Product outputs\parsed_output_400.json:
Processing C:\2nd Zara Product outputs\parsed_output_40

C:\Users\madan\AppData\Local\Temp\ipykernel_772\2859615535.py:13: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result_df = pd.concat([result_df, df], ignore_index=True, sort=False)


Processing C:\2nd Zara Product outputs\parsed_output_439.json:
Processing C:\2nd Zara Product outputs\parsed_output_440.json:
Processing C:\2nd Zara Product outputs\parsed_output_441.json:
Processing C:\2nd Zara Product outputs\parsed_output_442.json:
Processing C:\2nd Zara Product outputs\parsed_output_443.json:
Processing C:\2nd Zara Product outputs\parsed_output_444.json:
Processing C:\2nd Zara Product outputs\parsed_output_445.json:
Processing C:\2nd Zara Product outputs\parsed_output_446.json:
Processing C:\2nd Zara Product outputs\parsed_output_447.json:
Processing C:\2nd Zara Product outputs\parsed_output_448.json:
Processing C:\2nd Zara Product outputs\parsed_output_449.json:
Processing C:\2nd Zara Product outputs\parsed_output_450.json:
Processing C:\2nd Zara Product outputs\parsed_output_451.json:
Processing C:\2nd Zara Product outputs\parsed_output_452.json:
Processing C:\2nd Zara Product outputs\parsed_output_453.json:
Processing C:\2nd Zara Product outputs\parsed_output_45

C:\Users\madan\AppData\Local\Temp\ipykernel_772\2859615535.py:13: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result_df = pd.concat([result_df, df], ignore_index=True, sort=False)


Processing C:\2nd Zara Product outputs\parsed_output_489.json:
Processing C:\2nd Zara Product outputs\parsed_output_490.json:
Processing C:\2nd Zara Product outputs\parsed_output_491.json:
Processing C:\2nd Zara Product outputs\parsed_output_492.json:
Processing C:\2nd Zara Product outputs\parsed_output_493.json:
Processing C:\2nd Zara Product outputs\parsed_output_494.json:
Processing C:\2nd Zara Product outputs\parsed_output_495.json:
Processing C:\2nd Zara Product outputs\parsed_output_496.json:
Processing C:\2nd Zara Product outputs\parsed_output_497.json:
Processing C:\2nd Zara Product outputs\parsed_output_498.json:
Processing C:\2nd Zara Product outputs\parsed_output_499.json:
Processing C:\2nd Zara Product outputs\parsed_output_500.json:
Processing C:\2nd Zara Product outputs\parsed_output_501.json:
Processing C:\2nd Zara Product outputs\parsed_output_502.json:
Processing C:\2nd Zara Product outputs\parsed_output_503.json:
Processing C:\2nd Zara Product outputs\parsed_output_50

C:\Users\madan\AppData\Local\Temp\ipykernel_772\2859615535.py:13: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result_df = pd.concat([result_df, df], ignore_index=True, sort=False)


Processing C:\2nd Zara Product outputs\parsed_output_654.json:
Processing C:\2nd Zara Product outputs\parsed_output_655.json:
Processing C:\2nd Zara Product outputs\parsed_output_656.json:
Processing C:\2nd Zara Product outputs\parsed_output_657.json:
Processing C:\2nd Zara Product outputs\parsed_output_658.json:
Processing C:\2nd Zara Product outputs\parsed_output_659.json:
Processing C:\2nd Zara Product outputs\parsed_output_660.json:
Processing C:\2nd Zara Product outputs\parsed_output_661.json:
Processing C:\2nd Zara Product outputs\parsed_output_662.json:
Processing C:\2nd Zara Product outputs\parsed_output_663.json:
Processing C:\2nd Zara Product outputs\parsed_output_664.json:
Processing C:\2nd Zara Product outputs\parsed_output_665.json:
Processing C:\2nd Zara Product outputs\parsed_output_666.json:
Processing C:\2nd Zara Product outputs\parsed_output_667.json:
Processing C:\2nd Zara Product outputs\parsed_output_668.json:
Processing C:\2nd Zara Product outputs\parsed_output_66

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\2nd Zara Product outputs\\parsed_output_1318.json'

In [17]:
# Total rows = 125,000 approx
result_df

,name,price,sectionName,productId,color,keyword,category_id
0,OPEN-BACK DRESS - ZW COLLECTION,599000.0,WOMAN,311233350.0,Electric blue,open-back-dress---zw-collection,2290613
1,CONTRAST DRESS - ZW COLLECTION,1099000.0,WOMAN,311540730.0,Black,contrast-dress---zw-collection,2290613
2,WOOL BLEND TUXEDO BLAZER - ZW COLLECTION,1099000.0,WOMAN,308650046.0,Black,wool-blend-tuxedo-blazer---zw-collection,2290613
3,KNIT DRESS WITH OPEN BACK AND TINSEL,499000.0,WOMAN,319426886.0,Black,knit-dress-with-open-back-and-tinsel,2290613
4,ZW COLLECTION OPEN-BACK DRESS WITH HOOD,759000.0,WOMAN,309575116.0,Black,zw-collection-open-back-dress-with-hood,2290613
...,...,...,...,...,...,...,...
123701,,59000.0,WOMAN,290675904.0,214,,1881287
123702,EYELASH CURLER,89000.0,WOMAN,290431181.0,2-800,eyelash-curler,1881287
123703,,NaN,None,290137851.0,104,,1881287
123704,EYELASH CURLER,89000.0,WOMAN,290431182.0,2-800,eyelash-curler,1881287


In [18]:
result_df['price'] = result_df['price'] .fillna(0)
result_df['price'] = result_df['price'].round().astype(int)
result_df['productId'] = result_df['productId'].round().astype(int)

In [19]:
result_df['price'] = result_df['price']/100

In [20]:
# Check for NaN, None, null values, and empty strings in specific columns
columns_to_check = 'color'
text_to_replace = 'Missing'  # Replace with your desired text

contains_nan_or_none = result_df[columns_to_check].isnull() | result_df[columns_to_check].isna() | (result_df[columns_to_check] == '')
result_df[columns_to_check] = result_df[columns_to_check].fillna('Unknown')

if contains_nan_or_none.any():
    print(f"Column {columns_to_check } contains NaN, None, null values, or empty strings.")
else:
    print("Not Found")

Column color contains NaN, None, null values, or empty strings.


In [21]:
# Check for NaN, None, null values, and empty strings in specific columns
columns_to_check = 'sectionName'
text_to_replace = 'Missing'  # Replace with your desired text

contains_nan_or_none = result_df[columns_to_check].isnull() | result_df[columns_to_check].isna() | (result_df[columns_to_check] == '')
result_df = result_df[~contains_nan_or_none]

if contains_nan_or_none.any():
    print(f"Column {columns_to_check } contains NaN, None, null values, or empty strings.")
else:
    print("Not Found")

Column sectionName contains NaN, None, null values, or empty strings.


In [22]:
result_df

,name,price,sectionName,productId,color,keyword,category_id
0,OPEN-BACK DRESS - ZW COLLECTION,5990.0,WOMAN,311233350,Electric blue,open-back-dress---zw-collection,2290613
1,CONTRAST DRESS - ZW COLLECTION,10990.0,WOMAN,311540730,Black,contrast-dress---zw-collection,2290613
2,WOOL BLEND TUXEDO BLAZER - ZW COLLECTION,10990.0,WOMAN,308650046,Black,wool-blend-tuxedo-blazer---zw-collection,2290613
3,KNIT DRESS WITH OPEN BACK AND TINSEL,4990.0,WOMAN,319426886,Black,knit-dress-with-open-back-and-tinsel,2290613
4,ZW COLLECTION OPEN-BACK DRESS WITH HOOD,7590.0,WOMAN,309575116,Black,zw-collection-open-back-dress-with-hood,2290613
...,...,...,...,...,...,...,...
123700,EYE BRUSH #1,590.0,WOMAN,277546910,,eye-brush--1,1881287
123701,,590.0,WOMAN,290675904,214,,1881287
123702,EYELASH CURLER,890.0,WOMAN,290431181,2-800,eyelash-curler,1881287
123704,EYELASH CURLER,890.0,WOMAN,290431182,2-800,eyelash-curler,1881287


In [23]:
# Assuming 'result_df' is your DataFrame
# Create a boolean mask for rows where 'color' column contains numeric values within strings
numeric_color_mask = result_df['color'].str.contains(r'^\d+$')

# Filter the DataFrame using the mask to get rows with 'color' containing numeric values
numeric_color_df = result_df[numeric_color_mask]
numeric_color_df

,name,price,sectionName,productId,color,keyword,category_id
100,,35550.0,WOMAN,316750769,975,,2290613
205,TIE-DYE PRINT KNIT DRESS,3290.0,WOMAN,289200957,024,tie-dye-print-knit-dress,2290613
423,TIE-DYE PRINT KNIT DRESS,3290.0,WOMAN,289200957,024,tie-dye-print-knit-dress,2290795
570,FITTED KNIT OFF-THE-SHOULDER DRESS,3290.0,WOMAN,289200956,025,fitted-knit-off-the-shoulder-dress,2290795
1086,FITTED MIDI DRESS WITH PADDED SHOULDERS,2290.0,WOMAN,286938734,536,fitted-midi-dress-with-padded-shoulders,2290795
...,...,...,...,...,...,...,...
123522,LOOK,3290.0,KID,315817392,700,,2292266
123673,LOOK,2180.0,WOMAN,290169456,154,,1881297
123683,,590.0,WOMAN,295148535,725,,1881287
123684,,1290.0,WOMAN,290675379,215,,1881287


In [24]:
def replace_category(row):
    if row['price'] >= 35000 :
        return 'Full Set of Suit'
    else:
        return row['name']

result_df['name'] = result_df.apply(replace_category, axis=1)

C:\Users\madan\AppData\Local\Temp\ipykernel_772\38257103.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df['name'] = result_df.apply(replace_category, axis=1)


In [30]:
fd = result_df[result_df['price'] <= 35500]
fd = fd.sort_values(by='price', ascending=False)
fd.head(30)

,name,price,sectionName,productId,color,keyword,category_id
50453,Full Set of Suit,35170.0,MAN,316149412,407,textured-suit---limited-edition,2297784
38427,Full Set of Suit,35170.0,MAN,316149483,070,,2297657
58399,Full Set of Suit,35170.0,MAN,308060042,019,,2299411
58301,Full Set of Suit,35170.0,MAN,316149483,070,,2299477
58398,Full Set of Suit,35170.0,MAN,308060041,018,,2299411
39103,LOOK,34950.0,MAN,308556450,451,,2297664
38380,LOOK,34750.0,MAN,316571352,208,,2297657
39040,LOOK,34750.0,MAN,316571352,208,,2297664
38271,LOOK,34560.0,MAN,319183936,477,,2297657
38273,LOOK,34560.0,MAN,319183928,471,,2297657


In [31]:
result_df.to_csv('Zara_product_list.csv',index=False)